In [1]:
# import modin.pandas as pd
# import os
# os.environ["MODIN_ENGINE"] = "ray"
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot

In [13]:
import pandas as pd

In [14]:
cols = ['FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP','SATURACAO','DIARREIA','VOMITO',
                     'DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','TOMO_RES', 'VACINA_COV','HOSPITAL',
                     'UTI','SUPORT_VEN','RAIOX_RES','AMOSTRA','CLASSI_FIN','VACINA','NU_IDADE_N']
df_train = pd.read_csv('train.csv', usecols=cols)

In [15]:
from dtype_diet import report_on_dataframe, optimize_dtypes
optimized_df = report_on_dataframe(df_train,unit = "MB")
df_train_opt = optimize_dtypes(df_train,optimized_df)

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300980 entries, 0 to 1300979
Data columns (total 22 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   NU_IDADE_N  1300980 non-null  int64  
 1   FEBRE       1069610 non-null  float64
 2   TOSSE       1128895 non-null  float64
 3   GARGANTA    904415 non-null   float64
 4   DISPNEIA    1124930 non-null  float64
 5   DESC_RESP   1039725 non-null  float64
 6   SATURACAO   1077358 non-null  float64
 7   DIARREIA    886757 non-null   float64
 8   VOMITO      878227 non-null   float64
 9   VACINA      916185 non-null   float64
 10  HOSPITAL    1274497 non-null  float64
 11  UTI         1138453 non-null  float64
 12  SUPORT_VEN  1136018 non-null  float64
 13  RAIOX_RES   768899 non-null   float64
 14  AMOSTRA     1256816 non-null  float64
 15  CLASSI_FIN  1300980 non-null  int64  
 16  DOR_ABD     861941 non-null   float64
 17  FADIGA      910992 non-null   float64
 18  PERD_OLFT   866856 non

In [20]:
# Drop the values with nan in Tomo_RES and SATURACAO
df = df_train.dropna(subset=['TOMO_RES','SATURACAO']).reset_index(drop=True)

In [21]:
df.describe()

,NU_IDADE_N,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,VACINA,...,SUPORT_VEN,RAIOX_RES,AMOSTRA,CLASSI_FIN,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,VACINA_COV
count,748207.000000,685881.000000,702901.000000,633602.000000,717565.000000,694632.000000,748207.000000,632600.000000,627827.000000,618036.000000,...,727792.000000,572266.000000,734941.000000,748207.000000,624510.000000,647560.000000,627744.000000,627126.000000,748207.000000,629681.000000
mean,53.566081,1.529570,1.347416,2.009995,1.318228,1.451219,1.358143,2.029317,2.065888,4.949246,...,2.246947,5.393883,1.099409,4.626974,2.129798,1.854466,2.165969,2.169741,3.805597,2.712142
std,24.243998,1.047041,0.936092,1.253008,0.884871,1.003155,0.965404,1.154748,1.160073,3.587940,...,1.371884,2.288998,0.690422,0.698482,1.245570,1.242294,1.368282,1.380962,2.768121,2.764789
min,-9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,40.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,...,2.000000,4.000000,1.000000,4.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,57.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,...,2.000000,6.000000,1.000000,5.000000,2.000000,2.000000,2.000000,2.000000,4.000000,2.000000
75%,72.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,9.000000,...,2.000000,6.000000,1.000000,5.000000,2.000000,2.000000,2.000000,2.000000,6.000000,2.000000
max,136.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,5.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [22]:
from summarytools import dfSummary
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,NU_IDADE_N[int64],Mean (sd) : 53.6 (24.2)min < med < max:-9.0 < 57.0 < 136.0IQR (CV) : 32.0 (2.2),123 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACmklEQVR4nO3dQY6iQBSA4VeDGtQEYkx05wUmmYVLDzGHnSPMwhN4A3cmhmCkmxGR2U1caLdQMNSr/r893Sz+dENBPUxVVQK47lvfJwC8YtD3CbjKGDMWkVHDwy9VVb23eT5fHaE+YIwZz+fzn1EUzZocfzqdEmPML2JtD6E+NoqiaLbZbN7jOM7rHJimabjdbmfH43EkIoTaEkL9QBzH+WKxeGtw6Lj1k/niuJmCCoQKFQgVKnCN2oGyLIciEhljmhzO0tYDhNqyLMuGRVH8WK1WQRAEtVYMRFjaeoZQW3a5XAZhGE7W63W+XC6TOseytPUcoXZkOp2ytNUibqagAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVvH6Fabs6LbrfbsM3zgR0vQ7XdnFcURViW5fc8z3+LSJPn9WiZl6GKxeY8EZH9fj/b7XaT6/UadHFyqM/XUEWk+ea8JEl4g8kx3ExBBUKFCl7/69fIcr+ViKd7rgjVIbb7rUT83XNFqA6x2W8l4veeK0J1kMV+KxFP91xxMwUVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgWn30e1GCLBAAnPfBpqX58DtxkiwQAJ/3wYas+fA288RIIBEv757C9q758DbzJEggES/nnpGpXPgaNv3PVDBUKFCoQKFQgVKhAqVOj0yZTlwC+eLuGfzkK1HfjF0yXc6yxU24FfPF3Cvc5fSmk68IunS7jHzRRUIFSoQKhQwekXp1Gf5ZKgs/P/CdUjtkuCLs//J1SP2CwJuj7/n1A9ZPENAGeXBF8KNU3TsO4PPp/PoYhIlmXh4XCY/M/jtf7uPs87TdOwLMuB2H3jysafqqqeXq78BU4ua39HjL+YAAAAAElFTkSuQmCC"">",0(0.0%)
2,FEBRE[float64],1. 1.02. 2.03. nan4. 9.0,"395,291 (52.8%)280,214 (37.5%)62,326 (8.3%)10,376 (1.4%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACDklEQVR4nO3bwW0aURiF0R+EnLAZhJAogwIoIsWmCAqgD4SYRXC8IYtYUTZWwiTcB8w5FTxbnwa90dzJ5XIpSJi2PgDjITZiJlU1r6qX1gd5Am+Xy+Xc+hD3bLZarb50XbdsfZBH1/f9cTKZfBXcx2Zd1y232+15sVi8tj7MozqdTp93u93ycDi8VJXYPjCrqlosFq/r9fpb68M8uHnrA9w7FwRixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI2ZW9fMTmdYHeWT+f39n1vf9cbfbLcsnMv+k7/tjVb21Psc981n4/+Oz8D+YmPKR4oJAzJh+Rv3MNTaadZX1U3ujWFdZP92HMa2rvNppzAWBGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzGjmPI9+9/3KEYz5TO1a88GgRhTPmKGPtk8JbjaoHWVpRJDXL2uslRiqKHrqqe+uXIbXuoSIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI+bqdZWlEkMNWldZKjGEDQIx1lXE/P5k87Tipn6tqyymuLVp13XLzWZT73O+MSzjaWRaVTWfz7+3PgjPz0tdYsRGjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMRMq6rO5/On1gfh+U37vj/u93uLKW7OBoEY6ypiXBCI+QEhs8me3Io6uQAAAABJRU5ErkJggg=="">","62,326(8.3%)"
3,TOSSE[float64],1. 1.02. 2.03. nan4. 9.0,"515,773 (68.9%)178,975 (23.9%)45,306 (6.1%)8,153 (1.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACFUlEQVR4nO3bwW3aABiG4R+E0nKxhZAYgwEyRIftEAzAHgjhQ0lzoYdGVQ+0oAQ+2+R5JvilvHJk5G9yOp0KEqZ9H8DnITZiJlU1r6qnvg/hKq+n0+nY9xHvNVsul9+apln0fQiXdV23n0wm38ca3KxpmsXz8/OxbduXvo/h3w6Hw9fNZrPY7XZPVTXO2Kqq2rZ9Wa1WP/o+hovmfR/wEV4QiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI2YWdXvz1f6PoT/e4S/0azruv1ms1nUyD9f+Qy6rttX1Wvfd7yXz8LHZdSfhU9M+UjxgkDMUP+NjvrfBecNcl019hUR5w1uXfUIKyLOG+q6ys8wD8gLAjFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZiBjflG9It3NYgp3xjn6xxng0CMaZ8xFz7ZPOk4cOuWldZO3ELF9dV1k7cyrXrqsG8qTJeftQlRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERszFdZW1E7dy1brK2olbsEEgxrqKmL+fbJ5e3NWfdZUFFfc2bZpmsV6v623ON7RlPA9kWlU1n89/9n0Ij8+PusSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixESM2YsRGjNiImVZVHY/HL30fwuObdl233263FlTcnQ0CMdZVxHhBIOYXIbPJnllniRMAAAAASUVORK5CYII="">","45,306(6.1%)"
4,GARGANTA[float64],1. 2.02. 1.03. nan4. 9.0,"496,911 (66.4%)118,813 (15.9%)114,605 (15.3%)17,878 (2.4%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB8UlEQVR4nO3bQWrbUBiF0V/FhGYiYQxejBfRxXYR2o0x1qAJmaiDEhpjj4TfVVHPWcEDf8hIvNvN81yQ8G3tA/D/EBsxXVW9VtXL2gfhoY95nt/WPsSz7A6Hw4++7/drH4R70zRduq77uZXgdn3f70+n09swDO9rH4a/rtfr93Ec9+fz+aWqthFbVdUwDO/H4/HX2ofhzuvaB3gmLwjEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNm

In [43]:
df.groupby("TOMO_RES").count()

,NU_IDADE_N,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,VACINA,...,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,CLASSI_FIN,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,VACINA_COV
TOMO_RES,,,,,,,,,,,,,,,,,,,,,
1.0,321009,288700,296190,258319,304082,290404,321009,256628,252878,263100,...,307979,307848,203849,311254,321009,251411,266593,254353,254038,253806
2.0,29385,26165,27152,24183,28023,26862,29385,24247,24177,24466,...,28499,28501,21138,29118,29385,24042,24934,24042,23980,24932
3.0,21790,19628,20269,18424,20887,20086,21790,18446,18390,17601,...,21028,21097,15488,21599,21790,18332,18790,18280,18274,19082
4.0,4862,4402,4500,4137,4571,4401,4862,4094,4109,4023,...,4722,4730,3578,4836,4862,4081,4178,4058,4054,4214
5.0,39417,34945,36402,32253,37283,35535,39417,32484,32372,31814,...,38334,38476,28389,39173,39417,32163,33073,32145,32230,33635
6.0,266668,251773,256759,239416,259776,255753,266668,239735,239224,219532,...,253384,262870,240677,264463,266668,238066,242465,238333,238080,236846
9.0,65076,60268,61629,56870,62943,61591,65076,56966,56677,57500,...,62493,64270,59147,64498,65076,56415,57527,56533,56470,57166


In [26]:
import scipy.stats as ss

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [27]:
cat_cols = ['FEBRE','TOSSE','GARGANTA','DISPNEIA','DESC_RESP','SATURACAO','DIARREIA','VOMITO',
            'DOR_ABD','FADIGA','PERD_OLFT','PERD_PALA','TOMO_RES', 'VACINA_COV','HOSPITAL',
            'UTI','SUPORT_VEN','RAIOX_RES','AMOSTRA','TP_AMOSTRA','VACINA']
corr_matrix = pd.DataFrame(index=cat_cols, columns=['CLASSI_FIN'])
for col in cat_cols:
    corr_matrix.loc[col, 'CLASSI_FIN'] = cramers_v(df[col], df['CLASSI_FIN'])
    
# View the correlation matrix
print(corr_matrix)

C:\Users\ygorf\AppData\Local\Temp\ipykernel_8228\4088991509.py:12: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


           CLASSI_FIN
FEBRE        0.067212
TOSSE         0.06042
GARGANTA     0.064383
DISPNEIA     0.048882
DESC_RESP    0.033999
SATURACAO     0.07093
DIARREIA     0.052892
VOMITO       0.031453
DOR_ABD      0.016782
FADIGA        0.08206
PERD_OLFT    0.101555
PERD_PALA    0.103704
TOMO_RES     0.213127
VACINA_COV   0.109211
HOSPITAL          NaN
UTI          0.067592
SUPORT_VEN    0.09006
RAIOX_RES    0.096925
AMOSTRA           0.0
TP_AMOSTRA   0.030514
VACINA       0.046326


In [ ]:
for i in range(0,len(cols)):
    print(
        str(cols[i]) + ": " + str(np.corrcoef(cols[i], df["CLASSI_FIN"])[0][1] * 100) + " %"
    )

In [33]:
df = pd.get_dummies(df, columns=cols)

# Compute the correlation matrix
corr = df.corr()

# Plot the correlation matrix as a heatmap
sns.heatmap(corr, cmap='coolwarm', center=0, annot=True, fmt='.2f')

<AxesSubplot:>

In [12]:
df_train.groupby('CLASSI_FIN').count()

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,DOR_ABD,FADIGA,...,PERD_PALA,TOMO_RES,VACINA_COV,HOSPITAL,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,TP_AMOSTRA,VACINA
CLASSI_FIN,,,,,,,,,,,,,,,,,,,,,
1,12956,13836,10297,12274,11247,11503,9477,9562,9303,9755,...,9206,8522,15498,15004,12512,12570,8384,15029,14600,9603
2,27579,29568,21935,27032,26901,26060,21916,22408,21468,21849,...,21244,17891,29471,31263,29767,29470,24052,31417,30911,16694
3,4556,4821,3800,4746,4454,4451,3730,3762,3693,3820,...,3636,3564,5245,5588,4773,5044,3460,5550,5153,3066
4,309841,329570,265104,326747,304638,307657,260950,262502,256393,264701,...,252981,221608,328030,371651,320672,321864,235831,370883,346570,251061
5,714678,751100,603279,754131,692485,727687,590684,579993,571084,610867,...,579269,593785,673357,850991,770729,767070,497172,833937,785697,635761


In [13]:
df_train[df_train['CLASSI_FIN'] == 5].groupby("TOMO_RES").count()

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,DOR_ABD,FADIGA,...,PERD_PALA,VACINA_COV,HOSPITAL,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,TP_AMOSTRA,CLASSI_FIN,VACINA
TOMO_RES,,,,,,,,,,,,,,,,,,,,,
1.0,292937,304222,247406,307255,283244,300793,244708,239365,237010,256689,...,241225,265080,340026,324890,323775,204239,329792,303974,341583,268769
2.0,12830,13531,11126,13625,12662,13317,11081,10992,10851,11585,...,10894,12519,15423,14884,14829,10325,15294,14796,15464,12251
3.0,7611,7939,6829,7897,7444,7782,6797,6767,6719,6992,...,6693,7639,8916,8594,8546,5780,8823,8482,8952,7022
4.0,1888,1910,1597,1817,1680,1802,1566,1565,1531,1587,...,1505,2022,2320,2249,2226,1448,2316,2235,2330,1791
5.0,21176,22627,18623,22534,20513,22439,18744,18649,18410,19095,...,18556,21092,25744,24770,24998,17331,25643,24698,25819,19302
6.0,138682,142549,128041,142954,137124,140617,127065,126215,125348,129245,...,125838,132384,153671,146386,151841,136264,152677,145477,154506,131474
9.0,38855,40269,34866,40504,38450,39475,34590,34231,33956,35171,...,34142,38557,44954,43380,44462,39930,44680,42378,45131,40341


In [3]:
df_train[df_train['CLASSI_FIN'] == 5].groupby("RAIOX_RES").count()


,SEM_NOT,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,NU_IDADE_N,TP_IDADE,...,PERD_PALA,TOMO_RES,TOMO_OUT,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FNT_IN_COV,DELTA_UTI,ID
RAIOX_RES,,,,,,,,,,,,,,,,,,,,,
1.0,15134,15134,15134,13780,13780,15134,15134,15134,15134,15134,...,11707,10797,378,12793,15134,15134,15134,12794,15134,15134
2.0,96959,96959,96959,91709,91709,96959,96959,96959,96959,96959,...,75106,63627,1008,78080,96959,96959,96959,78091,96959,96959
3.0,11354,11354,11354,10363,10363,11354,11354,11354,11354,11354,...,8618,8646,270,9280,11354,11354,11354,9281,11354,11354
4.0,12343,12343,12343,11818,11818,12343,12343,12343,12343,12343,...,9872,9346,148,10241,12343,12343,12343,10241,12343,12343
5.0,31585,31585,31585,28640,28640,31585,31585,31585,31585,31585,...,24597,25658,2525,26190,31585,31585,31585,26195,31585,31585
6.0,249388,249388,249388,223692,223692,249388,249388,249388,249388,249388,...,205065,227457,9066,206929,249388,249388,249388,206945,249388,249388
9.0,80409,80409,80409,63976,63976,80409,80409,80409,80409,80409,...,63431,69786,1298,68354,80409,80409,80409,68361,80409,80409


In [21]:
from dtype_diet import report_on_dataframe, optimize_dtypes
optimized_df = report_on_dataframe(df_train,unit = "MB")
df_train_optimized = optimize_dtypes(df_train,optimized_df)
print(f'Original df memory:{df_train.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Original df memory:{df_train_optimized.memory_usage(deep=True).sum()/1024/1024} MB')


Original df memory:218.36529541015625 MB
Original df memory:53.350704193115234 MB
